In [18]:
import csv
import math
import numpy as np
import pandas as pd
import re
import nltk

from flair.data import Sentence
from scipy.spatial import distance
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LinearRegression, Ridge
from nltk import ngrams
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from flair.embeddings import ELMoEmbeddings
from nltk import edit_distance
from nltk.corpus import brown

from sklearn.preprocessing import PolynomialFeatures

c:\Users\bryan\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
def read_data_task1(filename, max=None, mode='orig', train=True, pad=True):
    """ method to read in data in csv format
        Arguments:
        filename        path to the file to read in
        max             maximum number of words per headline, needed for test set
    """
    texts = []
    labels = []
    ids = []
    indices = []
    tuples = []
    with open(filename, 'r', encoding='utf-8') as csv_file:
        reader = csv.reader(csv_file, quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True)
        next(reader)
        regex = re.compile('[^a-zA-Z\d\s\']') # pattern to remove non-alphanumeric characters
        max_num_words = 0
        cnt = 0
        for line in reader:
            # if cnt > 100:
            #     break
            ids.append(line[0].strip())
            if train:
                labels.append(line[4].strip())
            text = line[1]
            start = text.find("<")
            end = text.find("/>")
            orig = text[start:end + 2]
            edit = line[2]
            if mode == 'edit':
                text = text.replace(orig, edit)
            else:
                while " " in orig:
                    white = orig.find(" ")
                    new_orig = orig[white+1:]
                    text = text.replace(orig, new_orig)
                    orig = new_orig
            text = text.strip()
            text = text.strip('\n')
            text = text.lower()
            text = re.sub(regex, "", text)
            orig = orig.strip()
            orig = orig.lower()
            orig = re.sub(regex, "", orig)
            edit = edit.strip()
            edit = edit.lower()
            tup = (orig, edit)
            tuples.append(tup)
            edit = re.sub(regex, "", edit)
            text_list = re.split('\s+', text)
            if mode == 'edit':
                edit_index = text_list.index(edit)
            else:
                edit_index = text_list.index(orig)
            if len(text_list) > max_num_words:
                max_num_words = len(text_list)
            indices.append(edit_index)
            texts.append(text)
            cnt = cnt + 1
    if pad:
        if max:
            max_num_words = max # ignore longest essay if max was passed as argument"
        padded = []
        for i in range(len(texts)):
            t = texts[i].split()
            t.insert(0, "<")
            if len(t) > max_num_words: # truncate longer essays
                t = t[0:max_num_words+1]
            while len(t) <= max_num_words: # pad shorter essays
                t.append(">")
            t = " ".join(t)
            padded.append(t)
        texts = padded
    labels = np.array(labels, dtype=float)
    if train:
        return texts, labels, indices, ids, tuples
    else:
        return texts, indices, ids, tuples

In [19]:
def trunc_SVD(x, components=700):
    svd = TruncatedSVD(n_components=components)
    svd_x_train = svd.fit_transform(x)
    return svd_x_train

In [11]:
features, labels, idx, ids, tups = read_data_task1("semeval-2020-task-7-dataset/subtask-1/train.csv", mode='orig', pad=False)
ed_features, ed_labels, ed_idx, ed_ids, ed_tups = read_data_task1('semeval-2020-task-7-dataset/subtask-1/train.csv', mode='edit', pad=False)
f_dev, lab_dev, ind_dev, id_dev, t_dev = read_data_task1('semeval-2020-task-7-dataset/subtask-1/dev.csv', mode='orig', pad=False)
ed_fdev, ed_labdev, ed_inddev, ed_iddev, ed_tdev = read_data_task1('semeval-2020-task-7-dataset/subtask-1/dev.csv', mode='edit', pad=False)
f_test, ind_test, id_test, t_test = read_data_task1('semeval-2020-task-7-dataset/subtask-1/test.csv', train=False, mode='orig', pad=False)
ed_ftest, ed_indtest, ed_idtest, ed_ttest = read_data_task1('semeval-2020-task-7-dataset/subtask-1/test.csv', train=False, mode='edit', pad=False)